In [ ]:
import pandas as pd

keyword = "さい"
df = pd.read_json("grammar.json")
df_res = df[df.content.str.contains(keyword) | df.hiragana.str.contains(keyword) | df.meaning.str.contains(keyword)]
[df.loc[index].to_dict() for index in df_res.index]

In [2]:
import json
json.dumps("①")

'"\\u2460"'

In [5]:
print("\u2470")

⑰


In [29]:
import re

pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_pattern = re.compile(r'^[\d\s]+$')

In [33]:
import pandas as pd

example_sentences = []

with open("CS_Word_20240403_23.41.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_pattern.match(line.strip()):
            continue

        if pattern.match(line.strip()):
            matched = True
            sentence = {"content": pattern.sub(r"\1 ", line.strip())}
        else:
            if matched:
                sentence["hiragana"] = ""
                sentence["meaning"] = line.strip()
                print(sentence["content"])
                print(sentence["meaning"])
                if sentence["meaning"][1] == " ":
                    sentence["meaning"] = sentence["meaning"][2:]

                if sentence["meaning"][0] == "囉":
                    sentence["meaning"] = sentence["meaning"][1:]

                example_sentences.append(sentence)
            matched = False

with open("example_sentences.json", "w", encoding="utf-8") as fp:
    pd.DataFrame(example_sentences, dtype=str).fillna("").to_json(fp, orient="records", force_ascii=False, indent=4)

① この整理券は、 商品受け取りの際、必要です。
这张排号单在领取商品时需要用到。
② こちらの会議室をご利用になる際は、 受付で必要事項をご記入ください。
使用这间会议室时,请在前台登记必要事项。
③ アメリカの大統領は来日した際に、 わたしたちの大学でスピーチを行った。
译 美国总统访问日本时,在我们大学做了演讲。
① 工事関係者は工事を始めるに際して、 近所の住民にあいさつをして回った。
施工人员在开始施工前,走访问候了附近的住户。
② 当ショッピングサイトのご利用に際して、以下のご利用条件をよくお読みください。
使用本购物网站时,请仔细阅读以下使用条款。
③ 新しく事業を始めるにあたって、しっかりと準備をしようと思っております。
在新开展一项业务之际,我想做好充分的准备。
④ お二人の門出にあたりまして、 お祝いの言葉を申し上げます。
值此二位迎接新生活之际,送上我的祝福。
⑤ 日本で国際会議を開催するにあたり、関係各方面からの協力を得た。
在日本召开国际会议之际,得到了相关各方面的支持与配合。
① 山の頂上でワインを一口飲んだとたんに、 めまいがした。
在山顶上刚喝了一口葡萄酒就头晕了。
② 夫は結婚前は優しかったが、結婚したとたんに、 態度が変わった。
百丈夫在婚前很温柔,但是一结婚,态度就变了。
③ 国の母に電話をかけた。 母の声を聞いたとたん、 涙があふれてきた。
给家乡的妈妈打了电话。一听到妈妈的声音,我就热泪盈眶。
④ 僕が 「さよなら」 と言ったとたん、 彼女は走っていってしまった。
我刚一说“再见”,她就跑开了。
① 林さんは部屋に入ってきたかと思うと、 いきなり窓を全部開けた。
囉小林刚一进房间,就突然把窗戶全部打开了
② 赤ちゃんは今泣いたと思うと、もう笑っている。
深 小宝宝刚才还哭呢,现在又笑了起来。
③ やっと部屋が片付いたかと思ったら、子供たちがすぐまた散らかした。
好不容易收拾好房间,孩子们马上又弄乱了。
④ このごろは気温の差が大きい。 昨日は暑くなったかと思ったら、 今日は涼しい。
囉 最近溫差很大。昨天还很热,今天又很凉爽。
① 一郎はベッドに横になるかならないかのうちに、ぐっすり眠ってしまった。
太郎一躺到床上就酣然入睡了。
② わたしは夜が明けたか明けないかのうちに家を出て、空港へ向かった。
隔天一亮我就出了家门,前往